In [1]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import Adam
from matplotlib import pyplot as plt
import json
import os

c:\Users\Carlos\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Carlos\anaconda3\envs\pytorch\lib\site-packages\torch_geometric\typing.py:72: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not find module 'C:\Users\Carlos\anaconda3\envs\pytorch\Lib\site-packages\torch_scatter\_scatter_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "


In [3]:
# PyTorch implementations of Polynomial Kernels

def polynomial_kernel(X, Y, degree=2, gamma=1.0, coef0=1.0):
    """
    Polynomial kernel function.
    :param X: torch.Tensor of shape (n_samples_1, n_features)
    :param Y: torch.Tensor of shape (n_samples_2, n_features)
    :param degree: int, default=2
    :param gamma: float, default=1.0
    :param coef0: float, default=1.0
    :return: torch.Tensor of shape (n_samples_1, n_samples_2)
    """
    K = (gamma * torch.mm(X, Y.t()) + coef0) ** degree
    return K

def rbf_kernel(X, Y, gamma=None):

    """
    Rbf kernel function.
    :param X: torch.Tensor of shape (n_samples_1, n_features)
    :param Y: torch.Tensor of shape (n_samples_2, n_features)
    :param gamma: float or None, default=None
    :return: torch.Tensor of shape (n_samples_1, n_samples_2)
    """

    if gamma == None:
        gamma = 1.0/X.size(1) # gamma = 1/n_features
    
    # K(x, y) = exp(-gamma ||x-y||^2)
    d_XY = torch.cdist(X, Y, p=2) # pairwise distances between X and Y rows. Shape = (n_samples_1, n_samples_2)
    K = torch.exp(-gamma * d_XY ** 2)

    return K

def laplacian_kernel(X, Y, gamma=None):
    """
    Laplacian kernel function.
    :param X: torch.Tensor of shape (n_samples_1, n_features)
    :param Y: torch.Tensor of shape (n_samples_2, n_features)
    :param gamma: float or None, default=None
    :return: torch.Tensor of shape (n_samples_1, n_samples_2)
    """

    if gamma == None:
        gamma = 1.0/X.size(1) # gamma = 1/n_features
    
    # K(x, y) = exp(-gamma ||x-y||)
    d_XY = torch.cdist(X, Y, p=1) # pairwise distances between X and Y rows. Shape = (n_samples_1, n_samples_2)
    K = torch.exp(-gamma * d_XY)

    return K

def sigmoid_kernel(X, Y, gamma=1.0, coef0=1.0):
    """
    Sigmoid kernel function.
    :param X: torch.Tensor of shape (n_samples_1, n_features)
    :param Y: torch.Tensor of shape (n_samples_2, n_features)
    :param gamma: float, default=1.0
    :param coef0: float, default=1.0
    :return: torch.Tensor of shape (n_samples_1, n_samples_2)
    """
    K = torch.tanh(gamma * torch.mm(X, Y.t()) + coef0)
    return K


In [4]:
def binary_distance(X, Y):
    """Compute distance matrix between  rows of X, Y.

    d(x_i, y_j) = 1 if x_i == y_j, 0 in other case.

    for all rows x_i in X, y_j in Y

    """
    return (X.unsqueeze(1) == Y.unsqueeze(0)).all(-1).float()


def kernel(X, *args, **kwargs):
    """
    Compute similarity matrix of an array X using a variety of kernels
    Parameters:
        X = input data (torch.Tensor or numpy array)
    """
    # kernel parameters

    degree = kwargs.get("degree", None)
    gamma = kwargs.get("gamma", None)
    coef = kwargs.get("coef", None)
    kernel_type = kwargs.get("kernel_type", None)

    if kernel_type == "polynomial":


        # K(x,y) = (gamma * <x,y> + coef)^degree, for vectors x,y

        if (gamma != None) & (coef != None) & (degree != None):
            return polynomial_kernel(X = X, Y = X, degree = degree, gamma = gamma, coef0 = coef)
        else:
            return None

    elif kernel_type == "sigmoid":
 
        # K(x,y) = tanh(gamma * <x,y> + coef), for vectors x,y

        if (gamma != None) & (coef != None):
            return sigmoid_kernel(X = X, Y = X, gamma = gamma, coef0 = coef)
        else:
            return None

    elif kernel_type == "rbf":

        # K(x, y) = exp(-gamma ||x-y||^2)

        if (gamma != None):
            return rbf_kernel(X = X, Y = X, gamma = gamma)
        else:
            return None

    elif kernel_type == "laplacian":

        # K(x, y) = exp(-gamma ||x-y||_1)

        if (gamma != None):
            return laplacian_kernel(X = X, Y = X, gamma = gamma)
        else:
            return None

    else:
        return None


In [10]:
class SVBLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(SVBLinear, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        nn.init.orthogonal_(self.linear.weight)

    def forward(self, input):
        return self.linear(input)
    
    def singular_value_bounding(self):
        with torch.no_grad():
            # Descomposición SVD manual
            weight = self.linear.weight.data
            u, s, vh = torch.linalg.svd(weight, full_matrices=False)
            # Ajuste de valores singulares
            s = torch.clamp(s, min=0.9, max=1.1)
            # Reconstrucción de la matriz de pesos
            self.linear.weight.copy_(torch.mm(u, torch.mm(torch.diag(s), vh)))

class OrthogonalNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(OrthogonalNN, self).__init__()
        self.fc1 = SVBLinear(input_dim, 128)
        self.fc2 = SVBLinear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def apply_svb(self):
        self.fc1.singular_value_bounding()
        self.fc2.singular_value_bounding()

class Encoder_SLMVP(nn.Module):
    def __init__(self, encoder, kernel_parameters_X, optimizer, label_indep = False, kernel_parameters_Y = None):
        super().__init__()
        self.encoder = encoder
        self.kernel_parameters_X = kernel_parameters_X
        self.optimizer = optimizer
        self.label_indep = label_indep
        self.kernel_parameters_Y = kernel_parameters_Y
        self.criterion = nn.MSELoss()
    def encode(self, x):
        return self.encoder(x)
    
    def train_model(self, data_loader):
        self.train()
        
        epoch_losses = []

        for X, y in tqdm(data_loader):
            self.optimizer.zero_grad()
            Z = self.encode(X)

            # Calculation of similarity matrices
            K_X = kernel(X, **self.kernel_parameters_X)

            if self.label_indep: # data with different labels has not relationship (e.g: multiclass classification)
                K_Y = binary_distance(y.unsqueeze(1), y.unsqueeze(1))
            else:
                if self.kernel_parameters_Y == None:
                    self.kernel_parameters_Y = self.kernel_parameters_X
                K_Y = kernel(y, **self.kernel_parameters_Y)
            K_XY = K_X * K_Y
            D = torch.diag(torch.sum(K_XY, dim = 1))
            loss = torch.norm(torch.matmul(torch.matmul(torch.t(Z), D), Z) - torch.eye(Z.size(1)))
            epoch_losses.append(loss)

            loss.backward()
            self.optimizer.step()
        
        avg_loss = torch.mean(torch.Tensor(epoch_losses))
        return avg_loss.item()
    
    def test_model(self, data_loader):
        self.eval()
        epoch_losses = []

        with torch.no_grad():
            for X, y in data_loader:
                Z = self.encode(X)

                # Calculation of similarity matrices
                K_X = kernel(X, **self.kernel_parameters_X)
                if self.label_indep: # data with different labels has not relationship (e.g: multiclass classification)
                    K_Y = binary_distance(y.unsqueeze(1), y.unsqueeze(1))
                else:
                    if self.kernel_parameters_Y == None:
                        self.kernel_parameters_Y = self.kernel_parameters_X
                    K_Y = kernel(y, **self.kernel_parameters_Y)
                K_XY = K_X * K_Y
                D = torch.diag(torch.sum(K_XY, dim = 1))
                loss = torch.norm(torch.matmul(torch.matmul(torch.t(Z), D), Z) - torch.eye(Z.size(1)))
                epoch_losses.append(loss)
            
        avg_loss = torch.mean(torch.Tensor(epoch_losses))
        return avg_loss.item()

In [11]:
lr = 0.001 # learning rate
batch_size = 64
epochs = 1000
latent_size = 64 # latent space dimension

kernel_param_X = {

    "kernel_type": "rbf",
    "degree": 3,
    "gamma": 1e-2,
    "coef": 0.5e2

}

# kernel function parameters for Y_train / Y_test. If is equal to None, we use the same parameters.

kernel_param_Y = None

"""
kernel_param_Y = {

    "kernel_type": "polynomial",
    "degree": 3,
    "gamma": 1.,
    "coef": 1.

}
"""

label_indep = True # True for categoric labels, False for numerical labels

In [12]:


# train_data = pd.read_csv("/content/tfm_esteban/fashion_MNIST/fashion-mnist_train.csv")
# test_data = pd.read_csv("/content/tfm_esteban/fashion_MNIST/fashion-mnist_test.csv")

train_data = pd.read_csv("../fashion_MNIST/fashion-mnist_train.csv")
test_data = pd.read_csv("../fashion_MNIST/fashion-mnist_test.csv")
data = pd.concat([train_data, test_data], ignore_index = True)
# print(train_data.shape, test_data.shape, data.shape)
# data.head()
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=True, stratify=data["label"], random_state=20)
train_data, val_data = train_test_split(train_data, test_size=0.1, shuffle=True, stratify=train_data["label"], random_state=20)

# print(train_data["label"].value_counts(normalize=True))
# print(val_data["label"].value_counts(normalize=True))
# print(test_data["label"].value_counts(normalize=True))

X_train = np.array(train_data.drop("label", axis = 1, inplace=False))/255
X_train = torch.tensor(X_train, dtype = torch.float)
y_train = train_data["label"].values
y_train = torch.tensor(y_train, dtype = torch.float)
X_val = np.array(val_data.drop("label", axis = 1, inplace=False))/255
X_val = torch.tensor(X_val, dtype = torch.float)
y_val = val_data["label"].values
y_val = torch.tensor(y_val, dtype = torch.float)
X_test = np.array(test_data.drop("label", axis = 1, inplace=False))/255
X_test = torch.tensor(X_test, dtype = torch.float)
y_test = test_data["label"].values
y_test = torch.tensor(y_test, dtype = torch.float)

train_loader = DataLoader(list(zip(X_train, y_train)), shuffle = True, batch_size=batch_size)
val_loader = DataLoader(list(zip(X_val, y_val)), shuffle = True, batch_size=batch_size)
test_loader = DataLoader(list(zip(X_test, y_test)), shuffle = False, batch_size=batch_size)

input_size = X_train.size(1)

In [20]:
encoder = OrthogonalNN(input_size, latent_size)
optimizer = Adam(list(encoder.parameters()), lr = lr)

model = Encoder_SLMVP(encoder, kernel_param_X, optimizer, label_indep, kernel_param_Y)

In [ ]:
# # Datos de prueba
# input_dim = 10
# output_dim = 5
# model = OrthogonalNN(input_dim, output_dim)

# x = torch.randn(20, input_dim)
# target = torch.randn(20, output_dim)

# # Definición de la pérdida y el optimizador
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Entrenamiento
# num_epochs = 100
# for epoch in range(num_epochs):
#     optimizer.zero_grad()
#     outputs = model(x)
#     loss = criterion(outputs, target)
#     loss.backward()
#     optimizer.step()
    
#     # Aplicación periódica del método SVB
#     if epoch % 10 == 0:
#         model.apply_svb()

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



In [21]:
for epoch in range(epochs):
    print(f"Epoch: {epoch + 1}/{epochs}")
    avg_loss_train = model.train_model(train_loader)
    avg_loss_val = model.test_model(val_loader)

    print(f"avg_loss_train: {avg_loss_train}")
    print(f"avg_loss_val: {avg_loss_val}")

Epoch: 1/1000


100%|██████████| 788/788 [00:02<00:00, 347.43it/s]


avg_loss_train: 11.380729675292969
avg_loss_val: 7.249608993530273
Epoch: 2/1000


100%|██████████| 788/788 [00:02<00:00, 342.30it/s]


avg_loss_train: 7.157425403594971
avg_loss_val: 7.196472644805908
Epoch: 3/1000


...

avg_loss_train: 7.218732833862305
avg_loss_val: 7.234119892120361
Epoch: 174/1000


100%|██████████| 788/788 [00:03<00:00, 228.92it/s]


avg_loss_train: 7.24501371383667
avg_loss_val: 7.234248638153076
Epoch: 175/1000


 63%|██████▎   | 500/788 [00:02<00:01, 206.44it/s]


KeyboardInterrupt: 

In [22]:
model.test_model(test_loader)

7.216961860656738

In [23]:
for X, y in test_loader:
    Z = model.encode(X)
    break

In [24]:
Z

tensor([[-7.1741e-03, -8.8770e-03,  9.1274e-05,  ..., -3.7526e-03,
         -1.4313e-02, -1.3468e-02],
        [ 1.1667e-02,  1.1655e-01, -4.0782e-03,  ..., -3.6326e-02,
         -1.5243e-02, -2.2210e-02],
        [ 2.0192e-03, -4.9030e-03,  8.7791e-05,  ..., -4.9374e-03,
          5.9507e-04, -1.6853e-02],
        ...,
        [ 7.7304e-03, -2.8578e-02,  5.0133e-03,  ...,  8.6156e-02,
         -4.3978e-03,  1.4429e-02],
        [ 5.5903e-03,  2.3808e-01, -2.3548e-03,  ..., -1.5278e-02,
          1.6067e-01, -3.4925e-02],
        [-1.3213e-02, -1.0972e-02,  9.3293e-04,  ...,  2.4970e-02,
         -1.0905e-02,  6.0221e-04]], grad_fn=<AddmmBackward0>)

In [27]:
K_X = kernel(X, **kernel_param_X)
K_Y = binary_distance(y.unsqueeze(1), y.unsqueeze(1))

K_XY = K_X * K_Y
D = torch.diag(torch.sum(K_XY, dim = 1))
torch.matmul(torch.matmul(torch.t(Z), D), Z)

tensor([[ 1.5503e-02,  5.1791e-02, -6.5794e-04,  ..., -1.9561e-04,
         -1.5435e-02, -2.6198e-04],
        [ 5.1791e-02,  7.7149e-01, -5.1065e-03,  ...,  2.2171e-04,
          1.4602e-01, -7.0928e-02],
        [-6.5794e-04, -5.1065e-03,  3.9073e-03,  ...,  5.2087e-03,
         -1.6297e-02,  1.8563e-03],
        ...,
        [-1.9561e-04,  2.2172e-04,  5.2087e-03,  ...,  2.1537e-01,
         -2.8651e-02,  1.7989e-02],
        [-1.5435e-02,  1.4602e-01, -1.6297e-02,  ..., -2.8651e-02,
          6.5723e-01, -4.9386e-02],
        [-2.6198e-04, -7.0928e-02,  1.8563e-03,  ...,  1.7989e-02,
         -4.9386e-02,  4.4094e-02]], grad_fn=<MmBackward0>)